In [20]:
import requests
from datetime import datetime
from time import strftime
from json import dumps

def get_forecast(
    latitude: float,
    longitude: float,
    time: str,
):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m,precipitation_probability,precipitation,cloud_cover&timeformat=unixtime"
    response = requests.get(url)
    data = response.json()
    
    target_epoch = datetime.fromisoformat(time).timestamp()
    closest_index = min(range(len(data["hourly"]["time"])), key=lambda i: abs(data["hourly"]["time"][i] - target_epoch))
    
    return {
        "temperature": data["hourly"]["temperature_2m"][closest_index],
        "precipitation_probability": data["hourly"]["precipitation_probability"][closest_index],
        "precipitation": data["hourly"]["precipitation"][closest_index],
        "cloud_cover": data["hourly"]["cloud_cover"][closest_index],
    }

# Example for getting the current forecast for London
forecast = get_forecast(51.51, 0.13, strftime("%Y-%m-%d %H:%M"))
print(dumps(forecast, indent=4))

{
    "temperature": 14.3,
    "precipitation_probability": 1,
    "precipitation": 0.0,
    "cloud_cover": 100
}


In [21]:
from PBQA import DB, LLM
from time import strftime

class Agent:
    def __init__(self, db: DB, llm: LLM):
        self.db = db
        self.llm = llm
    
    def ask_weather(self, input: str):
        weather_query = self.llm.ask(
            input=input,
            pattern="weather",
            model="llama",
            external={"now": strftime("%Y-%m-%d %H:%M")}
        )
        
        print(f"Query:\n{dumps(weather_query, indent=4)}\n")
        
        forecast = get_forecast(**weather_query)
        forecast["temperature"] = f'{forecast["temperature"]}C'
        forecast["precipitation_probability"] = f'{forecast["precipitation_probability"]}%'
        forecast["precipitation"] = f'{forecast["precipitation"]}mm'
        forecast["cloud_cover"] = f'{forecast["cloud_cover"]}% of the sky'
        
        print(f"Forecast:\n{dumps(forecast, indent=4)}\n")
        
        return self.llm.ask(
            input=input,
            pattern="answer_json",
            model="llama",
            external={"json": dumps(forecast)}
        )

db = DB(path="db")
db.load_pattern("weather.yaml")
db.load_pattern("answer_json.yaml")

llm = LLM(db=db, host="192.168.0.137")
llm.connect_model(
    model="llama",
    port=8080,
    stop=["<|eot_id|>", "<|start_header_id|>"],
    temperature=0,
)

agent = Agent(db=db, llm=llm)
agent.ask_weather("Could I see the stars tonight?")

Query:
{
    "latitude": 51.51,
    "longitude": 0.13,
    "time": "2024-06-19 22:00"
}

Forecast:
{
    "temperature": "14.3C",
    "precipitation_probability": "1%",
    "precipitation": "0.0mm",
    "cloud_cover": "100% of the sky"
}



{'thought': "Unfortunately, the weather conditions don't look favorable for stargazing tonight. The cloud cover is 100% and there's no chance of precipitation, but the temperature is quite cool.",
 'answer': "Sorry, it doesn't look like you'll be able to see the stars tonight. The cloud cover is 100% and it's not going to be a clear night. Maybe try another night when the skies are clearer."}

Or more readibly,
    Sorry, it doesn't look like you'll be able to see the stars tonight. The cloud cover is 100% and it's not going to be a clear night. Maybe try another night when the skies are clearer.

Each model has its specific preferences in terms of formatting. In this case for example, I found out that llama3 prefers the unit of the value to be directly next to the value, without any whitespace between.